In [ ]:
import enchant
from tqdm import tqdm
import re
import pandas as pd
import seaborn as sns

In [ ]:
def parse_list(line:str):
    line = line.replace('[', '').replace(']', '')
    words = line.split(',')
    words = [w.replace("'", '').strip() for w in words if w != ""]
    return words

In [ ]:



if __name__ == '__main__':
    # load the original attributes to get the "class"
    with open('./../data/attributes_cleaned.txt', 'r') as file:
        content_og = file.readlines()

    id_to_class = dict()
    fails = 0
    for line in content_og:
        try:
            line = line.split(';')
            image_id = line[0].strip()
            class_id = line[1].strip()
            id_to_class[image_id] = class_id
        except Exception as e:
            fails += 1
    print('Failed to get the class to', fails, 'images')

    # load the extracted lens attributes
    with open('./../data/lens_attributes.txt', 'r') as file:
        content = file.readlines()

    # load english dictionary
    d = enchant.Dict("en_US")

    dataset = []

    output_file = open('./lens_attributes_dictionary.txt', 'w')
    fails = 0
    for line in tqdm(content):
        exploded = line.split(';')

        image_id = exploded[0]
        captions = exploded[2]
        attributes = exploded[3]

        # parse string list into python list
        parsed = parse_list(attributes)


        # words_in_dictionary = [w for w in parsed if d.check(w)]
        num_attributes_dictionary = 0
        num_attributes_split = 0

        words_in_dictionary = []
        for w in parsed:
            if d.check(w):
                words_in_dictionary.append(w)
                num_attributes_dictionary += 1
            # else:
            #     last_word = w.split(' ')[-1]
            #     if d.check(last_word):
            #         words_in_dictionary.append(f'{last_word}')
            #         num_attributes_split += 1
        # remove duplicates
        words_in_dictionary = list(set(words_in_dictionary))
        try:
            output_file.write(
                f'{image_id}; {id_to_class.get(image_id)}; {words_in_dictionary} \n'
            )
            dataset.append({
                'num_dictionary': num_attributes_dictionary,
                'num_split': num_attributes_split,
                'num_total': num_attributes_dictionary + num_attributes_split
            })

        except Exception as e:
            fails += 1
    print('Failed to trim the attributes of', fails, 'images')